# Scaling further with Dask Gateway

Dask can be deployed on distributed infrastructure, such as a an HPC system or a cloud computing system. There is a growing ecosystem of Dask deployment projects that faciliate easy deployment and scaling of Dask clusters on a wide variety of computing systems.

Within LEAP JupyterHub, we can use the `dask-gateway` to create dask clusters.


## Introduction to Dask Gateway

Dask Gateway helps you manage Dask clusters for multiple users in a scalable, secure, and resource-efficient way. It's designed to work well in environments like JupyterHub or shared cloud infrastructure.

Rather than manually setting up a Dask cluster, Dask Gateway automates the process, letting you spin up resources on-demand without worrying about underlying infrastructure details.



## When to Use Dask Gateway

Dask Gateway is useful when:

- You need to run Dask jobs on a shared cluster.
- You want to scale your computation dynamically, particularly in cloud or HPC environments.
- You prefer not to manage the underlying infrastructure manually (e.g., node management, worker allocation).

## Gateway Client


In [ ]:
from dask_gateway import Gateway

gateway = Gateway()

In [ ]:
options = gateway.cluster_options()
options

In [ ]:
cluster = gateway.new_cluster(options)
cluster

In [ ]:
client = cluster.get_client()
client

In [ ]:
cluster.scale(1)

In [ ]:
import dask.array as da

x = da.random.random((20_000, 20_000), chunks=(1000, 1000))
x

In [ ]:
y = x + x.T
z = y[::2, 5000:].mean(axis=1)
z

In [ ]:
%%time 
z.compute()

### Scaling our cluster on demand

In [ ]:
cluster.scale(4)

In [ ]:
%%time 
z.compute()

In [ ]:
cluster.scale(40)

In [ ]:
import xarray as xr

In [ ]:
ds = xr.open_dataset(
    "gs://cmip6/CMIP6/HighResMIP/MOHC/HadGEM3-GC31-HM/highresSST-present/r1i1p1f1/3hr/tas/gn/v20170831/", engine="zarr", chunks={}
)
ds

Passing `chunks={}` to `open_dataset()` works, but since we didn't tell dask how to split up (or chunk) the array, Dask will defer to the backend (`zarr`) to create chunks for our array. 

## Parallel and Lazy computation using `dask.array` with xarray


Xarray seamlessly wraps dask so all computation is deferred until explicitly requested. 

In [ ]:
z = ds.tas.mean(['lat', 'lon']).dot(ds.tas.T)
z

As you can see, `z` contains a dask array. This is true for all xarray built-in operations including subsetting

In [ ]:
z.isel(lat=0)

In [ ]:
%%time
z.compute()

## Other Distributed Systems for Dask

While Dask Gateway is a great tool for scaling, there are other ways to run Dask on distributed systems:

### HPC

#### Dask Jobqueue (https://jobqueue.dask.org/)

- `dask_jobqueue.PBSCluster`
- `dask_jobqueue.SlurmCluster`
- `dask_jobqueue.LSFCluster`
- etc.

#### Dask MPI (https://mpi.dask.org/)

- `dask_mpi.initialize`

### Cloud (https://coiled.io)

A managed cloud service for Dask that abstracts infrastructure concerns, making scaling even easier. 

- `coiled.Cluster`


#### Dask Kubernetes (https://kubernetes.dask.org/)

- `dask_kubernetes.KubeCluster`

#### Dask Cloud Provider (https://cloudprovider.dask.org)

- `dask_cloudprovider.FargateCluster`
- `dask_cloudprovider.ECSCluster`
- `dask_cloudprovider.ECSCluster`


